In [2]:
from dotenv import find_dotenv, load_dotenv
import os

load_dotenv(find_dotenv())
api_key = os.getenv("GROQ_API_KEY")

from llama_index.core import Settings
from llama_index.llms.groq import Groq

In [3]:
llm = Groq(model="llama-3.3-70b-versatile",
           api_key=api_key)

In [4]:
def calcular_imposto_renda(rendimento: float) -> str:
    """
    Calcula o imposto de renda com base no rendimento anual.
    
    Args:
        rendimento (float): Rendimento anual do indivíduo.
        
    Returns:
        str: O valor do imposto devido com base no rendimento
    """
    if rendimento <= 2000:
        return "Você está isento de pagar imposto de renda"
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"

### Convertendo Função em Ferramenta

In [5]:
from llama_index.core.tools import FunctionTool

In [6]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular Imposto de Renda",
    description=(
        "Calcula o imposto de renda com base no rendimento anual."
        "Argumento: rendimento (float)."
        "Retorna o valor do imposto devido de acordo com faixas de rendimento"
    )
)

In [7]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [8]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [9]:
from llama_index.core.agent import AgentRunner

In [10]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [11]:
response = agent_imposto.chat("""
    Qual é o imposto de renda devido por uma pessoa com rendimento
    anual de R$ 7.500?
    """)

Added user message to memory: 
    Qual é o imposto de renda devido por uma pessoa com rendimento
    anual de R$ 7.500?
    
=== Calling Function ===
Calling function: Calcular Imposto de Renda with args: {"rendimento": 7500}
=== Function Output ===
O imposto devido é de R$ 675.00, base em um rendimento de R$ 7500.00


Retrying llama_index.llms.openai.base.OpenAI._chat in 1.0 seconds as it raised InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}.


=== LLM Response ===
Lamento, mas não tenho como fornecer o valor exato do imposto de renda devido, pois depende de vários fatores, como a faixa de rendimento, deduções e outros. No entanto, posso dizer que o imposto de renda é calculado com base nas faixas de rendimento e alíquotas estabelecidas pela Receita Federal.

Se você quiser saber o valor exato do imposto de renda devido, recomendo consultar a tabela de imposto de renda da Receita Federal ou utilizar um simulador de imposto de renda online. Além disso, é importante lembrar que o imposto de renda pode variar de ano para ano, portanto, é importante verificar as informações atualizadas.


In [12]:
response = agent_imposto.chat("Quem foi Machado de Assis?")

Added user message to memory: Quem foi Machado de Assis?
=== LLM Response ===
Machado de Assis foi um escritor, poeta, contista e dramaturgo brasileiro, considerado um dos maiores nomes da literatura brasileira. Ele nasceu em 21 de junho de 1839, no Rio de Janeiro, e faleceu em 29 de setembro de 1908.

Machado de Assis é conhecido por suas obras-primas, como "Dom Casmurro", "Memórias Póstumas de Brás Cubas" e "Quincas Borba", que são consideradas algumas das melhores obras da literatura brasileira. Ele foi um dos principais representantes do Realismo e do Naturalismo no Brasil, e sua obra é caracterizada por uma prosa refinada, ironia e uma visão crítica da sociedade brasileira de sua época.

Além de sua obra literária, Machado de Assis também foi um jornalista, crítico literário e funcionário público. Ele foi um dos fundadores da Academia Brasileira de Letras e ocupou a cadeira número 23 da instituição.

Machado de Assis é considerado um dos mais importantes escritores brasileiros de 

### Trabalhando com arquivos - Nova Ferramenta Arxiv

In [13]:
import arxiv 

def consulta_artigos(titulo: str) -> str:
    """Consulta os artigos na base de dados ArXiv e retorna resultados formatados."""
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]
    
    return "\n\n".join(resultados)

In [14]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [15]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [18]:
agent = AgentRunner(agent_worker)
response = agent.chat("Me retorne artigos sobre LangChain na educação")

Added user message to memory: Me retorne artigos sobre LangChain na educação
=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "LangChain na educa\u00e7\u00e3o"}


/tmp/ipykernel_12074/1589355692.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
Título: Development and Testing of Retrieval Augmented Generation in Large Language Models -- A Case Study Report
Categoria: cs.CL
Link: http://arxiv.org/abs/2402.01733v1


Título: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integrated Web Application?
Categoria: cs.CR
Link: http://arxiv.org/abs/2308.01990v4


Título: Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations
Categoria: cs.CL
Link: http://arxiv.org/abs/2310.05421v1


Título: Poisoned LangChain: Jailbreak LLMs by LangChain
Categoria: cs.CL
Link: http://arxiv.org/abs/2406.18122v1


Título: Breast Ultrasound Report Generation using LangChain
Categoria: eess.IV
Link: http://arxiv.org/abs/2312.03013v1

=== LLM Response ===
Esses artigos podem ser úteis para entender como o LangChain pode ser aplicado na educação, mas é importante notar que a relação direta com a educação pode não ser explícita em todos os artigos. Além disso, 

### Usando o Tavily

In [19]:
import os 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [20]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [21]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [22]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

search


In [25]:
tavily_tool.search("Me retorne artigos científicos sobre LangChain", max_results=4)

[Document(id_='9d29d3ce-b426-46e1-ac11-ed0edd637269', embedding=None, metadata={'url': 'https://python.langchain.com/v0.1/docs/use_cases/question_answering/sources/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='The simplest way to do this is for the chain to return the Documents that were retrieved in each generation.', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='5c6bdd72-a10e-416f-b377-88f8b0ad3da5', embedding=None, metadata={'url': 'https://medium.com/@recogna.nlp/descomplicando-agentes-em-langchain-236e856ec687'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None,

In [26]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description="Busca artigos com Tavily sobre um determinado tópico"
)

In [27]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [28]:
agent = AgentRunner(agent_worker)

In [29]:
response = agent.chat("Me retorne artigos sobre LLM e LangChain")

Added user message to memory: Me retorne artigos sobre LLM e LangChain
=== Calling Function ===
Calling function: Tavily Search with args: {"query": "LLM e LangChain", "max_results": 10}
=== Function Output ===
[Document(id_='d6f7d86d-0b05-4dde-98c1-9a786efbc1df', embedding=None, metadata={'url': 'https://js.langchain.com/v0.1/docs/modules/chains/foundational/llm_chain/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='An LLMChain is a simple chain that adds some functionality around language models. It is used widely throughout LangChain, including in other chains and agents.', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'), Document(id_='5c568ddb-6bf1-4c69-b02a-fdcf8dcf5e6f', embedding=None, metadata={'url': 'https://python.langchai

In [30]:
print(response)

Aqui estão alguns artigos sobre LLM e LangChain:

1. "LLM Essentials: Getting Started with LangChain and OpenAI API" - Este artigo fornece uma introdução ao LangChain e ao OpenAI API, e como usá-los para construir aplicações de processamento de linguagem natural.
2. "LangChain: A Framework for Developing Applications Powered by Language Models" - Este artigo descreve o LangChain como um framework para desenvolver aplicações impulsionadas por modelos de linguagem.
3. "Designing LLM Applications with Knowledge Graphs and LangChain" - Este artigo explora como usar o LangChain e grafos de conhecimento para projetar aplicações de LLM.
4. "LangChain: A Suite of Products for Developing and Managing LLM Applications" - Este artigo descreve a suíte de produtos do LangChain para desenvolver e gerenciar aplicações de LLM.
5. "Understanding the Differences between LLM Chains and LLM Agent Executors in LangChain" - Este artigo explica as diferenças entre LLM Chains e LLM Agent Executors no LangChai

### Gerando Embeddings e Engine de Busca

In [31]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [32]:
url = "files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [33]:
url = "files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

### Gerar os Embeddings

In [34]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [35]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "intfloat/multilingual-e5-large"
)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/home/lucas/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [36]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [37]:
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

### Engine de Busca

In [38]:
from llama_index.core import StorageContext, load_index_from_storage

In [39]:
storage_context = StorageContext.from_defaults(
    persist_dir="artigo"
)
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
    persist_dir="tutorial"
)
tutorial_index = load_index_from_storage(storage_context)


In [40]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similarity_top_k=3, llm=llm)

In [41]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [42]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre LLM e LangChain."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "Fornece informações sobre casos de uso e aplicações em LLMs."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
]

In [43]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

In [44]:
response = agent_document.chat(
    "Quais as principais aplicações posso construir com LLM e LangChain?"
)

Added user message to memory: Quais as principais aplicações posso construir com LLM e LangChain?
=== Calling Function ===
Calling function: tutorial_engine with args: {"input": "Quais as principais aplica\u00e7\u00f5es posso construir com LLM e LangChain?"}
=== Function Output ===
As principais aplicações que você pode construir com LLM (Large Language Models) e LangChain incluem:

1. **Criação e aprimoramento de conteúdo**: Geração automática de texto, assistência na redação, tradução automática, resumo de textos, planejamento e roteiro de conteúdo, brainstorming e programação.
2. **Análise e organização de informações**: Análise de sentimento, extração de informações, classificação de textos, revisão técnica e extração de dados específicos de documentos grandes.
3. **Interação e automação**: Chatbots, perguntas e respostas, geração de respostas a perguntas com base em um corpus, e automação de tarefas de suporte.
4. **Casos de uso em produção**: Chatbots internos, extração de inform

In [45]:
response = agent_document.chat(
    "Quais as principais tendências em LangChain e LLM?"
)

Added user message to memory: Quais as principais tendências em LangChain e LLM?
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "Quais as principais tend\u00eancias em LangChain e LLM?"}
=== Function Output ===
As principais tendências em LangChain e LLM incluem o uso de modelos de código aberto, que permitem maior controle e personalização, além de serem mais acessíveis. Outra tendência é a capacidade de ajustar esses modelos a dados específicos, melhorando significativamente o desempenho em domínios específicos. Além disso, a importância de ter uma base sólida de dados para o uso eficaz dos LLMs também é uma tendência chave.
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "Quais as principais tend\u00eancias em LangChain e LLM?"}
=== Function Output ===
As principais tendências em LangChain e LLM incluem o uso de modelos de código aberto, que permitem maior controle e personalização, além de serem mais acessíveis. Além diss

### Agente ReAct

In [46]:
from llama_index.core.agent import ReActAgent

In [48]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    llm=llm
)

In [49]:
response = agent.chat("Quais as principais ferramentas usadas em LangChain?")

> Running step 5f906efe-a183-4fa7-b624-6d7e9323ac8e. Step input: Quais as principais ferramentas usadas em LangChain?
Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: artigo_engine
Action Input: {'input': 'Quais as principais ferramentas usadas em LangChain?'}
Observation: Não há menção às principais ferramentas usadas em LangChain no contexto fornecido. O contexto discute grandes modelos de linguagem (LLM), sua evolução, aplicações e comparação entre serviços proprietários e modelos de código aberto, mas não menciona LangChain ou suas ferramentas.
> Running step 71b1d48c-4215-461d-a16b-81a5452cafc5. Step input: None
Thought: A ferramenta artigo_engine não forneceu informações suficientes sobre as principais ferramentas usadas em LangChain. Vou tentar novamente com a ferramenta tutorial_engine.
Action: tutorial_engine
Action Input: {'input': 'Quais as principais ferramentas usadas em LangChain?'}
Observation: Não há 

In [50]:
response = agent.chat("Quais as principais tendências em LangChain que eu deveria estudar?")

> Running step 0682b8f6-b0ff-4e2b-bd14-23fea7e03b84. Step input: Quais as principais tendências em LangChain que eu deveria estudar?
Thought: O usuário está procurando por tendências em LangChain. Para fornecer uma resposta mais precisa, devo usar a ferramenta artigo_engine para obter informações sobre as principais tendências em LangChain.
Action: artigo_engine
Action Input: {'input': 'principais tendências em LangChain'}
Observation: Não há informações sobre LangChain nos textos fornecidos. No entanto, é possível identificar algumas tendências relacionadas aos grandes modelos de linguagem (LLMs) que podem estar relacionadas ao contexto de LangChain:

1. **Aumento da acessibilidade**: Com o lançamento de interfaces web simples, como o ChatGPT, os LLMs estão se tornando mais acessíveis a um público mais amplo.
2. **Melhoria dos dados de treinamento**: A coleta e análise de grandes volumes de dados de alta qualidade estão melhorando drasticamente o desempenho dos modelos.
3. **Avanços e